In [9]:
!pip install -q langchain_chroma langchain_openai langchain
!pip install -q datasets



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings()

persist_dir = './data'
vector_store = Chroma(persist_directory=persist_dir, embedding_function=embed_model)

retriever = vector_store.as_retriever()

In [3]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-4o-mini")


In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """
Answer the question in one sentence based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Loading data

##### im using datasets here because according to online sources, its faster esp if the dataset is bigger. 

In [6]:
import pandas as pd

# hugging face dataset
from datasets import Dataset

df = pd.read_csv("../qa.csv")
dataset = Dataset.from_pandas(df)

### Getting responses

In [41]:
def invoke_response(row):

    prompt = row['questions']
    answer = rag_chain.invoke(prompt)
    context = format_docs(retriever.invoke(prompt))
    return {"Responses": answer, "Generated_context": context}
    

In [42]:
dataset = dataset.map(invoke_response)
df_updated = dataset.to_pandas()


Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [66]:
df_updated.to_csv('output.csv', index=False)


In [43]:
df_updated.head(2)

,index,questions,answers,source_name,source,Responses,Generated_context
0,0,Who can be a member of the Boating Club?,Everyone is welcome to join! If you do not hav...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,"Community members, UC Santa Cruz students, fac...",Make sure to pay your membership dues and boat...
1,1,What classes does the boating center offer to ...,We offer all levels of sailing (dinghy and kee...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The boating center offers classes for beginner...,UC Santa Cruz Community Boating Center is a co...
